In [2]:
import os
import json
import gradio as gr
from dotenv import load_dotenv
from openai import OpenAI
from codex_template import run_codex

In [3]:
load_dotenv(override=True)
client = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
models = [
    'gpt-5.1-codex',    
    'gpt-5.1-codex-max',     
    'GPT-4 Turbo', 
    'gpt-5.1-chat-latest',
    'gpt-5',
    'gpt-4o'
]
languages = ['Python', 'C++', 'Rust', 'HTML', 'JavaScript', 'CSS', 'Go', 'Java', 'TypeScript']

In [4]:
from openai import api_key


def code_generator_tool(model, language, message):
    """Generates production-grade code using the unified run_codex function"""
    print(f'Code Generator tool called with model: {model}, language: {language}')

    full_prompt = f"""
    [System: You are an expert software engineering assistant. Generate production-grade, secure {language} code.]
    [User: {message}]
    
    Requirements:
    - Complete, functional code.
    - {language} style guides (PEP 8, etc.).
    - Security best practices.
    - Docstrings and usage examples.
    """
    return run_codex(full_prompt, model=model, api_key=api_key)

In [5]:
def code_explainer_tool(model, language, message):
    print(f'Code Explainer tool called with model: {model}, language: {language}')
    
    full_prompt = f"""
    [System: You are an expert code explainer for beginners. Explain this {language} code simply.]
    [User: {message}]
    
    Structure:
    1. Simple Summary
    2. Step-by-Step Breakdown (Analogies)
    3. Technical Details
    """
    
    # Use the unified caller!
    return run_codex(full_prompt, model=model, api_key=api_key)

In [6]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "code_generator_tool",
            "description": "Generates code based on requirements",
            "parameters": {
                "type": "object",
                "properties": {
                    "model": {"type": "string", "description": "Model to use for generation"},
                    "language": {"type": "string", "description": "Programming language"},
                    "message": {"type": "string", "description": "Code requirements"}
                },
                "required": ["model", "language", "message"]
            }
        }
    },
    {
        "type": "function",
        "function": {
            "name": "code_explainer_tool",
            "description": "Explains code snippets",
            "parameters": {
                "type": "object",
                "properties": {
                    "model": {"type": "string", "description": "Model to use for explanation"},
                    "language": {"type": "string", "description": "Programming language"},
                    "message": {"type": "string", "description": "Code to explain"}
                },
                "required": ["model", "language", "message"]
            }
        }
    }
]

In [7]:
def handle_tool_calls(tool_calls, selected_model, language):
    available_tools = {
        "code_generator_tool": code_generator_tool,
        "code_explainer_tool": code_explainer_tool
    }
    
    tool_responses = []
    
    for tool_call in tool_calls:
        function_name = tool_call.function.name
        function_args = json.loads(tool_call.function.arguments)
        
        function_args['model'] = selected_model
        function_args['language'] = language
        
        if function_name in available_tools:
            function_to_call = available_tools[function_name]
            print(f"Executing {function_name} using {selected_model}...")
            
            result = function_to_call(**function_args)
            
            tool_responses.append({
                "tool_call_id": tool_call.id,
                "role": "tool",
                "name": function_name,
                "content": str(result)
            })
    return tool_responses

In [8]:
def code_chatbot(target_model, language, message):
    """
    Main function. 
    1. Uses a standard 'Smart' model (gpt-5.1-chat-latest) to understand intent and route tools.
    2. The TOOLS themselves then use the 'target_model' (which might be the specialized Codex).
    """

    router_model = "gpt-5.1-chat-latest" 
    
    print(f" Router ({router_model}) processing request...")
    
    messages = [
        {"role": "system", "content": f"You are a coding assistant. Route the user's request to generate or explain {language} code."},
        {"role": "user", "content": message}
    ]
    
    try:
        response = client.chat.completions.create(
            model=router_model,
            messages=messages,
            tools=tools,
            tool_choice="auto",
            max_completion_tokens=500
        )
        
        if response.choices[0].message.tool_calls:
            tool_calls = response.choices[0].message.tool_calls
      
            tool_responses = handle_tool_calls(tool_calls, target_model, language)
            return tool_responses[0]["content"]
            
        else:
            return response.choices[0].message.content
    
    except Exception as e:
        return f"Error: {str(e)}\n\nMake sure your API key is valid."

In [9]:
with gr.Blocks(theme=gr.themes.Soft()) as demo:
    gr.Markdown("# Codex-Powered Assistant")
    gr.Markdown("Uses `run_codex` to seamlessly switch between Standard and Agentic models.")
    
    with gr.Row():
        with gr.Column():
            model_dropdown = gr.Dropdown(
                label=" Select Target Model",
                choices=models,
                value=models[5]
            )
            language_dropdown = gr.Dropdown(
                label=" Select Language",
                choices=languages,
                value="Python"
            )
            message_input = gr.Textbox(
                label=" Request",
                placeholder="Example: 'Create a snake game' or 'Explain this code...",
                lines=5
            )
            submit_btn = gr.Button("Submit", variant="primary")
        
        with gr.Column():
            response_output = gr.Markdown(label=" Response")
    
    submit_btn.click(
        fn=code_chatbot,
        inputs=[model_dropdown, language_dropdown, message_input],
        outputs=[response_output]
    )

if __name__ == "__main__":
    demo.launch(share=True)


* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://678c40e0f6c0d2c5f8.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


 Router (gpt-5.1-chat-latest) processing request...
 Router (gpt-5.1-chat-latest) processing request...
Executing code_explainer_tool using gpt-4o...
Code Explainer tool called with model: gpt-4o, language: Python
Generating code using gpt-4o...
 Router (gpt-5.1-chat-latest) processing request...
Executing code_generator_tool using gpt-4o...
Code Generator tool called with model: gpt-4o, language: HTML
Generating code using gpt-4o...
 Router (gpt-5.1-chat-latest) processing request...
Executing code_generator_tool using gpt-4o...
Code Generator tool called with model: gpt-4o, language: HTML
Generating code using gpt-4o...
